<a href="https://www.cognitiveclass.ai"><img src = "https://cognitiveclass.ai/wp-content/themes/bdu3.0/static/images/cc-logo.png" align = left></a>

<br>
<br>

--------------------
# Search for Extra Terrestrial Intelligence (SETI)
###  SETI Signal Classification on PowerAI with Multi GPU
<hr>
<br>

### Introduction
In this Notebook, we will use the famous [SETI Dataset](https://github.com/setiQuest/ML4SETI/) to build a Convolutional Neural Networks capable to perform signals classification. CNN will say, with some associated error, what type of signal is the presented input. In this notebook, you will use IBM PowerAI with multiple GPU to train the model.

### Project overview:
Each night, using the Allen Telescope Array (ATA) in northern California, the SETI Institute scans the sky at various radio frequencies, observing star systems with known exoplanets, searching for faint but persistent signals. The current signal detection system is programmed to search only for particular kinds of signals: narrow-band carrier waves. However, the detection system sometimes triggers on signals that are not narrow-band signals  (with unknown efficiency) and are also not explicitly-known radio frequency interference (RFI). There seems to be various categories of these kinds of events that have been observed in the past.

Our goal is to classify these accurately in real-time. This may allow the signal detection system to make better observational decisions, increase the efficiency of the nightly scans, and allow for explicit detection of these other signal types.

For more information refer to [SETI hackathon page](https://github.com/setiQuest/ML4SETI/).


Framing the radio signal data into spectrogram (a 2D visual representation), we can convert the problem into something akin to an image classification problem. CNN, will be run on the images which are the result of converting the signals to spectrogram.

### Training on Multi-GPU:
Today, many systems contains multiple GPUs for high performance computation. We can leverage this environments to run the training operation concurrently across multiple cards. One sample of these kind of environments is [IBM PowerAI](http://cocl.us/SETI-NIMBIX-PowerAI). In this notebook, we show you how to design and run your model on multiple GPUs.





In [1]:
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pickle
import time
#!sudo pip install sklearn
import os
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from six.moves import urllib
import sys
import tarfile


### Set the destination folder

In [2]:
### SET YOUR WORKING SPACE HERE! Use this folder to save intermediate results.
dataset_name = 'SETI_ds_64x128'
mydatafolder = "/tmp/SETI1_data/"
if os.path.exists(mydatafolder) is False:
    os.makedirs(mydatafolder)
print mydatafolder

train_dir = '/tmp/SETI1_train/'
if os.path.exists(train_dir) is False:
    os.makedirs(train_dir)
print train_dir

/tmp/SETI1_data/
/tmp/SETI1_train/


### Import dataset reader
The signals for this notebook, have been converted to spectogram images, and stored as 4 files.
The following cell will load a python code that help us to decode the binary file, and read the SETI dataset.

In [3]:
!wget -q --output-document  SETI.zip  https://ibm.box.com/shared/static/jhqdhcblhua5dx2t7ixwm88okitjrl6l.zip
!unzip -o SETI.zip
import SETI

Archive:  SETI.zip
  inflating: SETI.py                 
  inflating: __MACOSX/._SETI.py      


### Download data
The dataset exist and shared on IBM box. Running the following cell, you can download the dataset and extract it.

In [4]:
def maybe_download_and_extract():
    data_dir = "/tmp/SETI1_data/"
    DATA_URL =  'https://ibm.box.com/shared/static/qz33lcio9ip2j8qi2atxqs62gn3bnu2s.gz'
    dataset_name = 'SETI_ds_64x128'
    dest_directory = data_dir
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)
    filename = DATA_URL.split('/')[-1]
    filepath = os.path.join(dest_directory, filename)
    if not os.path.exists(filepath):
        def _progress(count, block_size, total_size):
            sys.stdout.write('\r>> Downloading %s %.1f%%' % (filename, float(count * block_size) / float(total_size) * 100.0))
        sys.stdout.flush()
        filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
    extracted_dir_path = os.path.join(dest_directory, dataset_name)
    if not os.path.exists(extracted_dir_path):
        tarfile.open(filepath, 'r:gz').extractall(dest_directory)
maybe_download_and_extract()

()
('Successfully downloaded', 'qz33lcio9ip2j8qi2atxqs62gn3bnu2s.gz', 2432541, 'bytes.')


### Load data SETI

In [5]:
ds_directory = mydatafolder + dataset_name
dataset = SETI.read_data_sets(ds_directory, one_hot=True, validation_size=0)
dataset.train.images.shape

Extracting /tmp/SETI1_data/SETI_ds_64x128/train-images-idx3-ubyte.gz
Extracting /tmp/SETI1_data/SETI_ds_64x128/train-labels-idx1-ubyte.gz
Extracting /tmp/SETI1_data/SETI_ds_64x128/test-images-idx3-ubyte.gz
Extracting /tmp/SETI1_data/SETI_ds_64x128/test-labels-idx1-ubyte.gz


(2000, 8192)

### Understanding the imported data

The imported data can be divided as follow:

- Training (dataset.train) >>  Use the given dataset with inputs and related outputs for training of NN. In our case, if you give an image that you know that represents a "class1", this set will tell the neural network that we expect a "class1" as the output.  
        - 694 signals (images)
        - dataset.train.images for inputs
        - dataset.train.labels for outputs
  
  
- Test (mnist.test) >> the model does not have access to this informations prior to the test phase. It is used to evaluate the performance and accuracy of the model against "real life situations". No further optimization beyond this point.  
        - 10,000 data points
        - dataset.test.images for inputs
        - dataset.test.labels for outputs
        
        
#### Labels
- Each image (spectrum of signal) in the dataset has been labeled from 1 to 4, representing:
    - squiggle
    - narrowband
    - noise
    - narrowbanddrd

## Network Parameters

We place the parameters on CPU.   
__Notice:__ This code is not optimal for single-GPU training due to the cost of copying parameters between CPU and GPU.

In [6]:
# Parameters
decay_rate=0.96  #decay every 1000 steps with a base of 0.96:
decay_steps=1000
learning_rate = 0.005
training_epochs = 10000
batch_size = 128
display_step = 100
moving_avg_decay = 0.9999     # The decay to use for the moving average.
use_fp16 = False

#check point directory
chk_directory = train_dir +'/save/'
checkpoint_path = chk_directory + 'model.ckpt'


num_classes = 4 # number of possible classifications for the problem
dropout = 0.50 # Dropout, probability to keep units

height = 64 # height of the image in pixels 
width = 128 # width of the image in pixels 
n_input = width * height # number of pixels in one image 
num_GPUs = 4

### How to use multi GPU for training?

PowerAI supports multi-GPU calculation.  Given multiple GPU cards, we run the SETI model on each GPU as following:

    1. Store all model parameters on the CPU.
    2. Put a copy of the SETI model on each GPU.
    3. Divide up a large batch of data across the GPUs, and feed each model replica with a unique batch of data.
    4. Compute the inference on each GPU
    5. Calculate the gradients on each GPU
    6. Wait until all GPUs finish processing of their batches
    7. Update model parameters synchronously on CPU

### Variable definitions

We define all variables using __tf.get_variable()__ in order to place the variables on CPU to be shares across multiple GPU for training. 

In [7]:
def _variable_on_cpu(name, shape, initializer):
    with tf.device('/cpu:0'):
        dtype = tf.float16 if use_fp16 else tf.float32
        var = tf.get_variable('v_on_cpu_' +name, shape, initializer=initializer, dtype=dtype)
    return var

In [8]:
def _variable_with_weight_decay(name, shape, stddev, wd):
    dtype = tf.float16 if use_fp16 else tf.float32
    var = _variable_on_cpu( name, shape,tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

### Inference

This function builds the graph as far as required for running the network forward to make predictions.

In [9]:
def inference(images):

    # conv1
    with tf.variable_scope('conv1') as scope: # a sample variable would be conv1/weights
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, 1, 32], stddev=0.1,  wd=0.0)
        conv = tf.nn.conv2d(images, kernel, strides=[1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [32], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)

    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights', shape=[5, 5, 32, 64],  stddev=0.1,  wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,  name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1], strides=[1, 4, 4, 1], padding='SAME', name='pool2')
    
    
    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 1024], stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [1024], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[1024, 256],  stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [256], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        #_activation_summary(local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [256, NUM_CLASSES],  stddev=1/256.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES], tf.constant_initializer(0.0))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
    #_activation_summary(softmax_linear)

    return softmax_linear

#### Loss function
loss function, adds the ops required to generate loss, to the inference graph. It calculates the average cross entropy loss across the batch.  

The total loss in calc_loss is defined as the cross entropy loss plus all of the weight decay terms (L2 loss).

In [10]:
def calc_loss(logits, labels):
    """Add L2Loss to all the trainable variables."""
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

As we want to run the CNN on multiple GPUs, we construct our model in a multi-tower fashion where each tower is assigned to a different GPU. The following function calculate the loss value for each tower.

In [11]:
# Calculate the total loss on a single tower running the SETI model.
def tower_loss(scope, images, labels):
    x_batch, labels = dataset.train.next_batch(batch_size,shuffle=True)
    images = tf.reshape(x_batch, [-1,height,width,1], name = 'image_batch')
    
    # Build inference Graph.
    logits = inference(images) # it builds a subgraph on twoer with all the operations, to use the shared variables.
    _=calc_loss(logits, labels) # calculate cross_entropy_mean and add it to collection

    # Assemble all of the losses for the current tower only.
    # 'losses' is the key for collection
    # scope is for e.g. 'tower_0'
    losses = tf.get_collection('losses', scope)

    # Calculate the total loss for the current tower.
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

#### Averaging Gradients 

Calculating the gradients on each tower, results in 4 group of gradients, stored in __tower_grads__. The following  function takes __tower_grads__, and calculates the average gradient for each shared variable across all towers.
Notice: that this function provides a synchronization point across all towers.

 - tower_grads: List of lists of (gradient, variable) tuples. The outer list is over individual gradients. The inner list is over the gradient calculation for each tower.

This function returns list of pairs of (gradient, variable) where the gradient has been averaged across all towers.


In [12]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
    # Note that each grad_and_vars looks like the following:
    #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
            # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

            # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

        # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

        # Keep in mind that the Variables are redundant because they are shared
        # across towers. So .. we will just return the first tower's pointer to
        # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

#### Training
 - When training a model, it is often recommended to lower the learning rate as the training progresses. 
This function applies an exponential decay function to a provided initial learning rate!

In [13]:
with tf.device('/cpu:0'):
    #These varibales are defined on CPU
    #with tf.Graph().as_default(), tf.device('/cpu:0'):
    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)

    # create learning_decay
    # Decay the learning rate exponentially based on the number of steps.
    lr = tf.train.exponential_decay( learning_rate,
                                     global_step,
                                     decay_steps,
                                     decay_rate, staircase=True )
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    optimizer = tf.train.GradientDescentOptimizer(lr)
    
    




 - A unique name for all operations within a tower. e.g. all operations in the first tower are prepended with tower_on_GPU0. We do it using name_scope(). It defines __operations__ with unique prefix name within each tower on the corresponding GPU. 
 - All variables are placed to the CPU and accessed via tf.get_variable (no variable on GPUs).

In [14]:
# Calculate the gradients for each model tower.
with tf.device('/cpu:0'):
    towers_grads = []
    with tf.variable_scope(tf.get_variable_scope()): # It is defined to share/reuse all the variables on CPU.
        for i in xrange(num_GPUs):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s-%d' % ('tower-on-GPU', i)) as nscope:
                    # read one batch for the GPU
                    #x_batch, y_batch = dataset.train.next_batch(batch_size,shuffle=True)
                    #image_batch = tf.reshape(x_batch, [-1,height,width,1], name = 'image_batch') 


                    # This function constructs the entire SETI model, calling inference.
                    # It shares the variables across all towers (e.g. conv1/weights).
                    # Calculate the loss for one tower of the SETI model. 
                    loss = tower_loss(nscope, _, _)

                    # For the next tower, when we call tower_loss() again, it calls inference(), and tries to create new variables.
                    # For example, it tries to create conv1/weights, again.
                    # We will reuse variables for the next tower. 
                    tf.get_variable_scope().reuse_variables()
                    
                    # Grads is a list over the gradient calculation for each tower.
                    # its shape looks like [(<tensor>,<variable>), (<tensor>,<variable>), ..]
                    # where, <tensor> is gradient value of <variable>
                    tower_grads = optimizer.compute_gradients(loss)
                    towers_grads.append(tower_grads)

    # calculate avg gradients on CPU                
    avg_grads = average_gradients(towers_grads)
    
    # Apply the gradients to adjust the shared variables.
    apply_gradient_op = optimizer.apply_gradients(avg_grads, global_step=global_step)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step, name= 'variable_averages')
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    # Group all updates to into a single train op.
    # we call train_op in learning process
    # tf.group Creates an op that groups multiple operations.
    # When this op finishes, all ops in input have finished. This op has no output.
    train_op = tf.group(apply_gradient_op, variables_averages_op)

You can check the constructed graph here:

In [15]:
# Helper functions for TF Graph visualization
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.

graph_def = tf.get_default_graph().as_graph_def()
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

#### Evaluation

In [16]:
import math
def evaluate_once():
    with tf.variable_scope(tf.get_variable_scope()):
        x_batch, y_test = dataset.test.next_batch(batch_size)
        tf.get_variable_scope().reuse_variables()
        image_test = tf.reshape(x_batch, [-1,height,width,1], name = 'image_test') 
        logits_test = inference(image_test)
        pred = tf.cast(logits_test, tf.float32)
        top_k_op = tf.nn.in_top_k(pred,tf.argmax(y_test,1) , 1)
        return top_k_op

In [17]:
def evaluate():
    with tf.name_scope("evaluation"):
        num_iter = int(math.ceil(dataset.test.num_examples / batch_size))
        true_count = 0  # Counts the number of correct predictions.
        total_sample_count = num_iter * batch_size
        step = 0
        while step < num_iter:
            x_batch, y_test = dataset.test.next_batch(batch_size)
            predictions = sess.run([evaluate_once()])
            true_count += np.sum(predictions)
            #print true_count
            step += 1

        # Compute precision @ 1.
        precision = true_count*1.0 / total_sample_count
        return precision
        #print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

### Create checkpoint directory

In [18]:
directory = os.path.dirname(chk_directory)
try:
    os.stat(directory)
    ckpt = tf.train.get_checkpoint_state(chk_directory)
    print ckpt
except:
    os.mkdir(directory) 

None


## Training

In [19]:
# Initializing the variables
init = tf.global_variables_initializer()
loss_values = []

X_test = dataset.test.images
y_test = dataset.test.labels

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
sess.run(init)
saver = tf.train.Saver(tf.global_variables())

# load previously trained model if appilcable
ckpt = tf.train.get_checkpoint_state(chk_directory)
if ckpt:
    print "loading model: ",ckpt.model_checkpoint_path
    saver.restore(sess, ckpt.model_checkpoint_path)

In [20]:
num_examples = dataset.train.num_examples
print 'Training size:',num_examples
print 'Total epochs:', training_epochs
print 'Batch size:', batch_size
total_batch = int(num_examples / batch_size)
print 'Total batchs per epoch:',total_batch
# Training cycle
tr_start = time.time()
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_accuracy = 0.

    # Loop over all batches in one epoch
    
    for step in range(total_batch):
        t_start = time.time()
        _, loss_value = sess.run([train_op, loss])
        duration = time.time() - t_start
        assert not np.isnan(loss_value), 'Model diverged with loss = NaN'
        
    # Display model every 1 epochs
    if epoch >= 0 and epoch % 100 == 0:
        plr = sess.run(lr)
        g_step = sess.run(global_step)
        loss_values.append(loss_value)
        sec_per_batch = duration / num_GPUs
        test_eval = evaluate()
        print "Epoch:", '%04d' % (epoch+1) , ", g_step:", '%04d' % (g_step) , ", Batch (sec)=" , "{:.5f}".format(sec_per_batch) , ", lr=", "{:.9f}".format(plr), ", cost=", "{:.9f}".format(loss_value) , ", pred=", "{:.9f}".format(test_eval)

print("Wall Time:","{:.1f}".format(time.time() - t_start), "sec")
print("Optimization Finished!")
print ("model saved to {}".format(checkpoint_path))
saver.save(sess, checkpoint_path, global_step = (epoch+1)*step)

Training size: 2000
Total epochs: 10000
Batch size: 128
Total batchs per epoch: 15
Epoch: 0001 , g_step: 0015 , Batch (sec)= 0.01633 , lr= 0.005000000 , cost= 22.785049438 , pred= 0.240885417
Epoch: 0101 , g_step: 1515 , Batch (sec)= 0.01635 , lr= 0.004800000 , cost= 21.555334091 , pred= 0.263020833
Epoch: 0201 , g_step: 3015 , Batch (sec)= 0.01681 , lr= 0.004423680 , cost= 20.456338882 , pred= 0.248697917
Epoch: 0301 , g_step: 4515 , Batch (sec)= 0.01644 , lr= 0.004246733 , cost= 19.483507156 , pred= 0.268229167
Epoch: 0401 , g_step: 6015 , Batch (sec)= 0.01671 , lr= 0.003913789 , cost= 18.608243942 , pred= 0.234375000
Epoch: 0501 , g_step: 7515 , Batch (sec)= 0.01643 , lr= 0.003757237 , cost= 17.828575134 , pred= 0.243489583
Epoch: 0601 , g_step: 9015 , Batch (sec)= 0.01668 , lr= 0.003462669 , cost= 17.123037338 , pred= 0.248697917
Epoch: 0701 , g_step: 10515 , Batch (sec)= 0.01685 , lr= 0.003324162 , cost= 16.491041183 , pred= 0.252604167
Epoch: 0801 , g_step: 12015 , Batch (sec)= 0

Epoch: 7501 , g_step: 112515 , Batch (sec)= 0.01680 , lr= 0.000051683 , cost= 9.341686249 , pred= 0.251302083
Epoch: 7601 , g_step: 114015 , Batch (sec)= 0.01652 , lr= 0.000047631 , cost= 9.336914062 , pred= 0.238281250
Epoch: 7701 , g_step: 115515 , Batch (sec)= 0.01664 , lr= 0.000045726 , cost= 9.332340240 , pred= 0.252604167
Epoch: 7801 , g_step: 117015 , Batch (sec)= 0.01649 , lr= 0.000042141 , cost= 9.327918053 , pred= 0.259114583
Epoch: 7901 , g_step: 118515 , Batch (sec)= 0.01668 , lr= 0.000040455 , cost= 9.323818207 , pred= 0.263020833
Epoch: 8001 , g_step: 120015 , Batch (sec)= 0.01600 , lr= 0.000037284 , cost= 9.320018768 , pred= 0.263020833
Epoch: 8101 , g_step: 121515 , Batch (sec)= 0.01646 , lr= 0.000035792 , cost= 9.316585541 , pred= 0.242187500
Epoch: 8201 , g_step: 123015 , Batch (sec)= 0.01639 , lr= 0.000032986 , cost= 9.313372612 , pred= 0.253906250
Epoch: 8301 , g_step: 124515 , Batch (sec)= 0.01692 , lr= 0.000031667 , cost= 9.310407639 , pred= 0.240885417
Epoch: 840

ValueError: GraphDef cannot be larger than 2GB.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.plot([np.mean(loss_values[i:i+5]) for i in range(len(loss_values))])
plt.show()


## Want to learn more?

Running deep learning programs usually needs a high performance platform. PowerAI speeds up deep learning and AI. Built on IBM's Power Systems, PowerAI is a scalable software platform that accelerates deep learning and AI with blazing performance for individual users or enterprises. The PowerAI platform supports popular machine learning libraries and dependencies including Tensorflow, Caffe, Torch, and Theano. You can download a [free version of PowerAI](http://cocl.us/SETI-NIMBIX-PowerAI).

### Authors

<div class="teacher-image" style="    float: left;
    width: 115px;
    height: 115px;
    margin-right: 10px;
    margin-bottom: 10px;
    border: 1px solid #CCC;
    padding: 3px;
    border-radius: 3px;
    text-align: center;"><img class="alignnone wp-image-2258 " src="https://ibm.box.com/shared/static/tyd41rlrnmfrrk78jx521eb73fljwvv0.jpg" alt="Saeed Aghabozorgi" width="178" height="178" /></div>
#### Saeed Aghabozorgi

[Saeed Aghabozorgi](https://ca.linkedin.com/in/saeedaghabozorgi), PhD is Sr. Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients’ ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

